In [11]:
all_methods= ['Two-hybrid','PCA','Co-fractionation', 'Biochemical Activity',  'Protein-RNA', 'Affinity Capture-Luminescence', 'Far Western', 'Reconstituted Complex', 'Affinity Capture-Western', 'Co-crystal Structure', 'Co-purification', 'Proximity Label-MS', 'Protein-peptide', 'Co-localization', 'Affinity Capture-RNA',  'Affinity Capture-MS', 'FRET']


binary_methods = ["Two-hybrid",'PCA', 'Biochemical Activity', 'Affinity Capture-Luminescence', 'Reconstituted Complex', 'Co-crystal Structure', 'FRET']

In [12]:
def get_data(dir):
    import json
    import os
    database = {}
    for filename in os.listdir(dir):
        dicti = {}
        f = os.path.join(dir, filename)
        with open (f) as s:
            data = s.read()
            data = data.replace('\'', '\"')
            dicti = json.loads(data)
            name = filename.split('.')[0]
            database[name] = dicti['ij']        
    print(database.keys())
    return database

In [20]:
def get_interactors(dict):
    import pandas as pd
    import numpy as np
    interactome_all = {}
    interactome_bi = {}
    index = {'CSNK1A1': '1452', 'CSNK1A1L': '122011', 'CSNK1D': '1453', 'CSNK1E': '1454'}
    all_methods= ['Two-hybrid','PCA','Co-fractionation', 'Biochemical Activity',  'Protein-RNA', 'Affinity Capture-Luminescence', 'Far Western', 'Reconstituted Complex', 'Affinity Capture-Western', 'Co-crystal Structure', 'Co-purification', 'Proximity Label-MS', 'Protein-peptide', 'Co-localization', 'Affinity Capture-RNA',  'Affinity Capture-MS', 'FRET']
    binary_methods = ["Two-hybrid",'PCA', 'Biochemical Activity', 'Affinity Capture-Luminescence', 'Reconstituted Complex', 'Co-crystal Structure', 'FRET']
    for protein in dict.keys():
        interactome_all[protein] = {}
        interactome_bi[protein] = {}
        for bait1, prey1 in dict[protein].items():
            proteinid = index[protein]
            if bait1 == proteinid:
                for bait2, prey2 in prey1.items():
                    count1 = 0
                    count2 = 0
                    for bait3,prey3 in prey2.items():
                        count1 = prey3 + count1
                        if bait3 in binary_methods:
                            count2 = prey3 + count2
                    interactome_all[protein][bait2]= count1
                    interactome_bi[protein][bait2] = count2
            else:
                for bait2, prey2 in prey1.items():
                    count3=0
                    count4=0
                    for bait3, prey3 in prey2.items():
                        count3 = prey3 + count3
                        if bait3 in binary_methods:
                            count4 = prey3 + count4
                    interactome_all[protein][bait1] = count3
                    interactome_bi[protein][bait1] = count4
    df1 = pd.DataFrame.from_dict(interactome_all)
    df2 = pd.DataFrame.from_dict(interactome_bi)
    df1 = df1.replace(np.nan, 0)
    df2 = df2.replace(np.nan, 0)
    return(df1, df2)


In [21]:
dir='biogrid_data'
database = get_data(dir)
interactome_all, interactome_bi = get_interactors(database)
interactome_all.to_csv('interactome_all.csv')
interactome_bi.to_csv('interactome_bi.csv')
pd.DataFrame.head(interactome_all)

dict_keys(['CSNK1A1', 'CSNK1E', 'CSNK1D', 'CSNK1A1L'])


CSNK1A1  CSNK1E  CSNK1D  CSNK1A1L
10010          1.0     0.0     1.0       0.0
100506658      1.0     2.0     0.0       0.0
10112          1.0     0.0     1.0       1.0
102724631      1.0     0.0     0.0       0.0
10410          1.0     0.0     0.0       0.0

In [15]:
def intersection(a, b):
    return list(set(a) & set(b))